In [1]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
max_limit = 6
budget_diff_penalizer = 1


def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)
    
"""
def compare_solutions(data, data2):
    results = {}
    
    for case_id, case_data in data.items():
        n_qubits = case_data["hyperparams"]["n_qubits"]
        cont_sol = case_data.get("continuous_variables_solution", {})
        exact_sol = case_data.get("exact_solution", {})
        qaoa_sol = case_data.get("qaoa_solution", {})
        budget = case_data["hyperparams"]["budget"]

        
        un_cont_sol = data2[case_id]["continuous_variables_solution_unconstrained"]
        allocation2 = un_cont_sol["allocation"]
        used_budget = 0
        for stock, weight in allocation2.items():
            used_budget += weight * data2[case_id]["hyperparams"]["prices_now"][stock]
        unconstrained_cont_value = un_cont_sol["value"]
        unconstrained_cont_penalty = budget_diff_penalizer*un_cont_sol["left_overs"]
        
        cont_value = cont_sol["value"]
        cont_penalty = budget_diff_penalizer*cont_sol["left_overs"]

        exact_budgets = exact_sol.get("result_with_budget", [])[0]
        exact_value = exact_budgets.get("objective_value", 0)
        exact_penalty = budget_diff_penalizer*exact_budgets["difference"]

        qaoa_value = qaoa_sol.get("objective_values", [])[-1]
        qaoa_budgets = qaoa_sol.get("result_with_budget", [])[-1]
        qaoa_penalty = budget_diff_penalizer*qaoa_budgets["difference"]

        results[case_id] = {
            "continuous": {"value": cont_value, "penalty": cont_penalty},
            "continuous_unconstrained": {"value": unconstrained_cont_value, "penalty": unconstrained_cont_penalty},
            "exact": {"value": exact_value, "penalty": exact_penalty},
            "qaoa": {"value": qaoa_value, "penalty": qaoa_penalty},
            "n_qubits": n_qubits,
            "budget": budget
        }
    
    return results
"""

def compare_solutions(data, data2, data3, data4):
    results = {}
    
    for case_id, case_data in data.items():
        n_qubits = case_data["hyperparams"]["n_qubits"]
        cont_sol = data3[case_id]["continuous_variables_solution"]
        exact_sol = data4[case_id]["exact_solution"]
        qaoa_sol = case_data["qaoa_solution"]
        budget = case_data["hyperparams"]["budget"]
        un_cont_sol = data2[case_id]["continuous_variables_solution_unconstrained"]
        
        allocation2 = un_cont_sol["allocation"]
        used_budget = 0
        for stock, weight in allocation2.items():
            used_budget += weight * data2[case_id]["hyperparams"]["prices_now"][stock]
        unconstrained_cont_value = un_cont_sol["value"]
        unconstrained_cont_penalty = budget_diff_penalizer*un_cont_sol["left_overs"]
        
        cont_value = cont_sol["value"]
        cont_penalty = budget_diff_penalizer*cont_sol["left_overs"]

        exact_budgets = exact_sol.get("result_with_budget", [])[0]
        exact_value = exact_budgets.get("objective_value", 0)
        exact_penalty = budget_diff_penalizer*exact_budgets["difference"]

        qaoa_value = qaoa_sol.get("objective_values", [])[-1]
        qaoa_budgets = qaoa_sol.get("result_with_budget", [])[-1]
        qaoa_penalty = budget_diff_penalizer*qaoa_budgets["difference"]

        results[case_id] = {
            "continuous": {"value": cont_value, "penalty": cont_penalty},
            "continuous_unconstrained": {"value": unconstrained_cont_value, "penalty": unconstrained_cont_penalty},
            "exact": {"value": exact_value, "penalty": exact_penalty},
            "qaoa": {"value": qaoa_value, "penalty": qaoa_penalty},
            "n_qubits": n_qubits,
            "budget": budget
        }
    
    return results


def process_results(results):
    
    case_labels = []

    processed_data = {}
    
    # Process data for plotting
    for case_id, case_data in results.items():
        
        vals = [case_data["continuous"]["value"], case_data["exact"]["value"], case_data["qaoa"]["value"], case_data["continuous_unconstrained"]["value"]]
        max_val = max(vals)
        min_val = min(vals)
        obj_range = max_val - min_val
        # Normalize objective values to [0,1] scale
        cont_obj = (case_data["continuous"]["value"] - min_val)/obj_range
        exact_obj = (case_data["exact"]["value"]- min_val)/obj_range
        qaoa_obj = (case_data["qaoa"]["value"]- min_val)/obj_range
        unconstrained_cont_obj = (case_data["continuous_unconstrained"]["value"]- min_val)/obj_range
        
        # Calculate budget utilization percentage
        budget = case_data["budget"]
        cont_budget_pct = 100 - case_data["continuous"]["penalty"] / budget * 100
        ex_budget_pct = 100 - case_data["exact"]["penalty"] / budget * 100
        q_budget_pct = 100 - case_data["qaoa"]["penalty"] / budget * 100
        u_budget_pct = 100 - case_data["continuous_unconstrained"]["penalty"] / budget * 100

        processed_data[case_id] = {
            "continuous": {"value": cont_obj, "budget_pct": cont_budget_pct},
            "exact": {"value": exact_obj, "budget_pct": ex_budget_pct},
            "qaoa": {"value": qaoa_obj, "budget_pct": q_budget_pct},
            "continuous_unconstrained": {"value": unconstrained_cont_obj, "budget_pct": u_budget_pct},
            "n_qubits": case_data["n_qubits"],
            "budget": budget
        }
    
    count_continuous = 0
    count_exact = 0
    count_qaoa = 0
    count_unconstrained = 0

    for case_id, case_data in processed_data.items():
        # If value is between 0.95 and 1 and budget_pct between 95-105, count it as a success
        if case_data["continuous"]["value"] >= 0.95 and 95 <= case_data["continuous"]["budget_pct"] <= 105:
            count_continuous += 1
        if case_data["exact"]["value"] >= 0.95 and 95 <= case_data["exact"]["budget_pct"] <= 105:
            count_exact += 1
        if case_data["qaoa"]["value"] >= 0.95 and 95 <= case_data["qaoa"]["budget_pct"] <= 105:
            count_qaoa += 1
        if case_data["continuous_unconstrained"]["value"] >= 0.95 and 95 <= case_data["continuous_unconstrained"]["budget_pct"] <= 105:
            count_unconstrained += 1
    
    count_budget_satisfied_cont = 0
    count_budget_satisfied_exact = 0
    count_budget_satisfied_qaoa = 0
    count_budget_satisfied_unconstrained = 0

    for case_id, case_data in processed_data.items():
        # If budget_pct is between 95-105, count it as a success
        if 95 <= case_data["continuous"]["budget_pct"] <= 105:
            count_budget_satisfied_cont += 1
        if 95 <= case_data["exact"]["budget_pct"] <= 105:
            count_budget_satisfied_exact += 1
        if 95 <= case_data["qaoa"]["budget_pct"] <= 105:
            count_budget_satisfied_qaoa += 1
        if 95 <= case_data["continuous_unconstrained"]["budget_pct"] <= 105:
            count_budget_satisfied_unconstrained += 1
    
    count_obj_value_satisfied_cont = 0
    count_obj_value_satisfied_exact = 0
    count_obj_value_satisfied_qaoa = 0
    count_obj_value_satisfied_unconstrained = 0

    for case_id, case_data in processed_data.items():
        # If value is between 0.95 and 1, count it as a success
        if case_data["continuous"]["value"] >= 0.95:
            count_obj_value_satisfied_cont += 1
        if case_data["exact"]["value"] >= 0.95:
            count_obj_value_satisfied_exact += 1
        if case_data["qaoa"]["value"] >= 0.95:
            count_obj_value_satisfied_qaoa += 1
        if case_data["continuous_unconstrained"]["value"] >= 0.95:
            count_obj_value_satisfied_unconstrained += 1
    
    return {
        "count_continuous": count_continuous,
        "count_exact": count_exact,
        "count_qaoa": count_qaoa,
        "count_unconstrained": count_unconstrained,
        "count_budget_satisfied_cont": count_budget_satisfied_cont,
        "count_budget_satisfied_exact": count_budget_satisfied_exact,
        "count_budget_satisfied_qaoa": count_budget_satisfied_qaoa,
        "count_budget_satisfied_unconstrained": count_budget_satisfied_unconstrained,
        "count_obj_value_satisfied_cont": count_obj_value_satisfied_cont,
        "count_obj_value_satisfied_exact": count_obj_value_satisfied_exact,
        "count_obj_value_satisfied_qaoa": count_obj_value_satisfied_qaoa,
        "count_obj_value_satisfied_unconstrained": count_obj_value_satisfied_unconstrained
    }

    
    

In [2]:
def main():
            # Define the lambda folders to check
    lambda_folders = {
        0.001: "lambda_0001",
        0.01: "lambda_001",
        0.1: "lambda_01",
        1.0: "lambda_1",
        0.9: "lambda_09",
        10.0: "lambda_10",
        100.0: "lambda_100",
        1000.0: "lambda_1000"
    }
    #lambda_folders = {1.0 : "lambda_1"} # For testing purposes, only use lambda_1 folder
    
    method = "cmaes"
    base_path = f"../results/{method}_hubo_results/"
    
    # Dictionary to store the best results and lambdas for each case
    data = {}
    best_lambdas = {}

    best_exact_data = {}
    best_exact_lambdas = {}
    
    # Process each lambda folder
    for lambda_val, lambda_folder in lambda_folders.items():
        folder_path = os.path.join(base_path, lambda_folder)
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist. Skipping.")
            continue
            
        # Load all files in this lambda folder
        print(f"Processing lambda={lambda_val} folder...")
        files = [file for file in os.listdir(folder_path) if "portfolio_optimization_" in file]
        
        # Process each file
        for file in files:
            file_path = os.path.join(folder_path, file)
            file_data = load_json(file_path)
            
            # For each case in the file
            for case_id, case_data in file_data.items():
                if case_id in ["37", "12", "75"]:
                    if lambda_val == 1.0:
                        data[case_id] = case_data
                        best_lambdas[case_id] = lambda_val
                    else:
                        continue
                    
                # Get expectation value
                expectation_val = case_data['qaoa_solution']['objective_values'][-1]
                
                # If we haven't seen this case before, or if this result is better
                if case_id not in data or expectation_val > data[case_id]['qaoa_solution']['objective_values'][-1]:
                    data[case_id] = case_data
                    best_lambdas[case_id] = lambda_val

                if 'exact_solution' in case_data:
                    exact_expectation_val = case_data['exact_solution']['smallest_eigenvalues'][0]
                    if case_id not in best_exact_data:
                        best_exact_data[case_id] = case_data
                        best_exact_lambdas[case_id] = lambda_val
                        #print(f"New exact solution for case {case_id} with lambda {lambda_val}: {exact_expectation_val}")
                    elif exact_expectation_val < best_exact_data[case_id]['exact_solution']['smallest_eigenvalues'][0]:
                        
                        #print(f"Updated exact solution for case {case_id} with lambda {lambda_val}: {exact_expectation_val}")
                        #print("Difference in expectation value:", 
                        #      best_exact_data[case_id]['exact_solution']['smallest_eigenvalues'][0] - exact_expectation_val)
                        best_exact_data[case_id] = case_data
                        best_exact_lambdas[case_id] = lambda_val
    
    data2 = load_json("../results/classical_unconstrained/filtered_portfolio_optimization_results.json")
    data3 = load_json("../results/classical_constrained/filtered_portfolio_optimization_results.json")
    data4 = load_json("../results/exact_eigensolver/filtered_portfolio_optimization_results.json")
    
    print(f"Loaded {len(data)} cases.")
    results = compare_solutions(data, data2, data3, data4)
    
    data = process_results(results)
    
    return data

data = main()

print(json.dumps(data, indent=4))

Processing lambda=0.001 folder...
Processing lambda=0.01 folder...
Processing lambda=0.1 folder...
Processing lambda=1.0 folder...
Processing lambda=0.9 folder...
Processing lambda=10.0 folder...
Processing lambda=100.0 folder...
Processing lambda=1000.0 folder...
Loaded 100 cases.
{
    "count_continuous": 13,
    "count_exact": 12,
    "count_qaoa": 10,
    "count_unconstrained": 12,
    "count_budget_satisfied_cont": 74,
    "count_budget_satisfied_exact": 100,
    "count_budget_satisfied_qaoa": 15,
    "count_budget_satisfied_unconstrained": 70,
    "count_obj_value_satisfied_cont": 26,
    "count_obj_value_satisfied_exact": 12,
    "count_obj_value_satisfied_qaoa": 80,
    "count_obj_value_satisfied_unconstrained": 25
}
